In [27]:
import os

import tensorflow as tf
from keras.models import Input, Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import Conv2D, Dropout, MaxPooling2D, UpSampling2D, concatenate

In [28]:
model = None

BATCH_SIZE = 64
INPUT_SIZE = 224
TARGET_SIZE = 256
N_EPOCHS = 10

train_data_path = r'D:\Datasets\DRIVE\training'
validation_data_path = r'D:\Datasets\DRIVE\test'

In [ ]:
def read_model_from_file(path=r'models\unet.h5'):
    model = load_model(path)
    return model

model = read_model_from_file()
model.summary()

In [29]:
def define_model(INPUT_SIZE):
    model = None

    input_shape = (TARGET_SIZE, TARGET_SIZE, 1)
    inputs = Input(input_shape)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Dropout(0.2)(conv1)
    conv1 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Dropout(0.2)(conv3)
    conv3 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(drop5))
    merge6 = concatenate([drop4, up6], axis=3)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(256, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv6))
    merge7 = concatenate([conv3, up7], axis=3)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

    up8 = Conv2D(128, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv7))
    merge8 = concatenate([conv2, up8], axis=3)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv8)

    up9 = Conv2D(64, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv8))
    merge9 = concatenate([conv1, up9], axis=3)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)
    model.compile(optimizer=Adam(lr=1e-6), loss='binary_crossentropy', metrics=['accuracy'])
    model.save(r'models\unet.h5')
    model.summary()
    # save the model as json string
    json_string = model.to_json()
    open(r'models\unet.json', 'w').write(json_string)
    
    return model

model = define_model(INPUT_SIZE)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
conv2d_49 (Conv2D)              (None, 256, 256, 64) 640         input_3[0][0]                    
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, 256, 256, 64) 0           conv2d_49[0][0]                  
__________________________________________________________________________________________________
conv2d_50 (Conv2D)              (None, 256, 256, 64) 36928       dropout_11[0][0]                 
__________________________________________________________________________________________________
max_poolin

In [31]:
def train_model(model, train_data_path, train_steps, n_epochs):
    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255, 
        validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        train_data_path,
        target_size=(TARGET_SIZE, TARGET_SIZE),
        color_mode='grayscale',
        batch_size=BATCH_SIZE, 
        subset='training')
    
    val_generator = datagen.flow_from_directory(
        validation_data_path,
        target_size=(TARGET_SIZE, TARGET_SIZE),
        color_mode='grayscale',
        batch_size=BATCH_SIZE, 
        subset='validation')
    
    history = model.fit_generator(train_generator, steps_per_epoch=32, 
                                  epochs=n_epochs, validation_data=val_generator, validation_steps=32)
    
    return history

train_model(model, train_data_path, 32, N_EPOCHS)

Found 32 images belonging to 4 classes.
Found 4 images belonging to 4 classes.
Epoch 1/10


ValueError: Error when checking target: expected conv2d_72 to have 4 dimensions, but got array with shape (32, 4)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()